In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os

# Set CUDA device before importing TensorFlow
os.environ["CUDA_VISIBLE_DEVICES"] = "6"  # Make only GPU 5 visible to TensorFlow

# Configure TensorFlow to use GPU memory growth to avoid taking all GPU memory
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Configure TensorFlow to use memory growth
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print(f"Using GPU: {tf.config.list_physical_devices('GPU')}")
    except Exception as e:
        print(f"Error configuring GPU: {e}")
else:
    print("No GPU found, using CPU instead")

# Verify CUDA configuration
print(f"TensorFlow is using devices: {tf.config.list_physical_devices()}")
print(f"TensorFlow CUDA available: {tf.test.is_built_with_cuda()}")
print(f"TensorFlow GPU available: {tf.test.is_gpu_available()}")

class NoiseRobustClassifier:
    def __init__(self, name, model_builder, transition_matrix=None, use_correction=True):
        """
        A wrapper for noise-robust classifiers.
        
        Args:
            name: Name of the classifier for reporting
            model_builder: Function that returns a TF/Keras model
            transition_matrix: The noise transition matrix if known
            use_correction: Whether to use transition matrix correction
        """
        self.name = name
        self.model_builder = model_builder
        self.transition_matrix = transition_matrix
        self.use_correction = use_correction
        self.model = None
        self.history = None
    
    def forward_correction_loss(self, y_true, y_pred):
        """Forward correction loss using transition matrix"""
        # Handle case when y_true is one-hot encoded
        if len(tf.shape(y_true)) > 1:
            y_true = tf.argmax(y_true, axis=1)
        
        # Get predicted probabilities
        pred_probs = tf.nn.softmax(y_pred)
        
        # Apply transition matrix correction
        T = tf.constant(self.transition_matrix, dtype=tf.float32)
        corrected_probs = tf.matmul(pred_probs, T)
        
        # Calculate cross-entropy loss with corrected probabilities
        loss = tf.nn.sparse_categorical_crossentropy(y_true, corrected_probs)
        
        return tf.reduce_mean(loss)
    
    def fit(self, X_train, S_train, X_val, S_val, epochs=30, batch_size=128):
        """
        Train the classifier on noisy data and validate.
        
        Args:
            X_train: Training features
            S_train: Noisy training labels
            X_val: Validation features
            S_val: Noisy validation labels
        """
        # Use explicit device placement to ensure operations run on the specified GPU
        with tf.device('/device:GPU:0'):  # This refers to the first (and only) visible GPU in the environment
            # Build a new model (to reset weights for multiple runs)
            self.model = self.model_builder()
            
            # Configure the loss function based on whether we use correction
            if self.transition_matrix is not None and self.use_correction:
                loss = self.forward_correction_loss
            else:
                # If no transition matrix provided or correction disabled, use standard CE
                loss = 'sparse_categorical_crossentropy'
            
            # Compile the model
            self.model.compile(
                optimizer='adam',
                loss=loss,
                metrics=['accuracy']
            )
            
            # Train the model
            self.history = self.model.fit(
                X_train, S_train,
                validation_data=(X_val, S_val),
                epochs=epochs,
                batch_size=batch_size,
                verbose=1,  # Show progress for monitoring
                callbacks=[
                    tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss',
                        patience=5,
                        restore_best_weights=True
                    )
                ]
            )
        
        return self
    
    def evaluate(self, X_test, Y_test):
        """
        Evaluate the classifier on clean test data.
        
        Args:
            X_test: Test features
            Y_test: Clean test labels
            
        Returns:
            Accuracy on the test set
        """
        # Make predictions
        with tf.device('/device:GPU:0'):
            y_pred = self.model.predict(X_test, verbose=0)
        y_pred_classes = np.argmax(y_pred, axis=1)
        
        # Calculate accuracy
        accuracy = np.mean(y_pred_classes == Y_test)
        
        return accuracy

def evaluate_classifier(classifier, Xtr, Str, Xts, Yts, n_runs=3):
    """
    Evaluate a classifier multiple times with different random splits.
    
    Args:
        classifier: A NoiseRobustClassifier instance
        Xtr: Training features
        Str: Noisy training labels
        Xts: Test features
        Yts: Clean test labels
        n_runs: Number of evaluation runs
        
    Returns:
        mean_acc, std_acc: Mean and standard deviation of test accuracies
    """
    accuracies = []
    
    for i in range(n_runs):
        # Random seed for reproducibility but different splits
        seed = 42 + i
        
        # Split data into train/validation
        X_train, X_val, S_train, S_val = train_test_split(
            Xtr, Str, test_size=0.2, random_state=seed
        )
        
        print(f"Run {i+1}/{n_runs} - Training {classifier.name}...")
        
        # Train the classifier
        classifier.fit(X_train, S_train, X_val, S_val)
        
        # Evaluate on test set
        accuracy = classifier.evaluate(Xts, Yts)
        accuracies.append(accuracy)
        print(f"Run {i+1}/{n_runs} - Test accuracy: {accuracy:.4f}")
    
    # Calculate mean and standard deviation
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    
    return mean_acc, std_acc

def run_evaluations(datasets, classifiers, n_runs=3):
    """
    Run evaluations for multiple classifiers on multiple datasets.
    
    Args:
        datasets: Dict of {dataset_name: (Xtr, Str, Xts, Yts)}
        classifiers: List of NoiseRobustClassifier instances
        n_runs: Number of runs with different random splits
        
    Returns:
        Dictionary of results {dataset_name: {classifier_name: (mean_acc, std_acc)}}
    """
    results = {}
    
    for dataset_name, (Xtr, Str, Xts, Yts) in datasets.items():
        print(f"\n=== Evaluating on dataset: {dataset_name} ===")
        dataset_results = {}
        
        for classifier in classifiers:
            print(f"\n--- Evaluating classifier: {classifier.name} ---")
            mean_acc, std_acc = evaluate_classifier(classifier, Xtr, Str, Xts, Yts, n_runs)
            dataset_results[classifier.name] = (mean_acc, std_acc)
            print(f"Final results - Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
        
        results[dataset_name] = dataset_results
    
    return results

def visualize_results(results, title="Classifier Performance Comparison"):
    """
    Create bar plots to visualize classifier performance across datasets.
    
    Args:
        results: Dictionary of results from run_evaluations
        title: Plot title
    """
    # Determine dimensions
    n_datasets = len(results)
    n_classifiers = len(next(iter(results.values())))
    
    # Set up the figure
    plt.figure(figsize=(12, 5 * n_datasets))
    
    # For each dataset
    for i, (dataset_name, dataset_results) in enumerate(results.items()):
        # Create subplot
        plt.subplot(n_datasets, 1, i+1)
        
        # Extract data for plotting
        classifiers = list(dataset_results.keys())
        accuracies = [dataset_results[clf][0] for clf in classifiers]  # mean
        errors = [dataset_results[clf][1] for clf in classifiers]      # std
        
        # Create bar plot
        bars = plt.bar(classifiers, accuracies, yerr=errors, capsize=10)
        
        # Add labels and grid
        plt.title(f"{dataset_name} - Test Accuracy")
        plt.ylabel("Accuracy")
        plt.ylim(0, 1.0)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Add accuracy values on top of bars
        for bar, acc in zip(bars, accuracies):
            plt.text(bar.get_x() + bar.get_width()/2, 
                     bar.get_height() + 0.01, 
                     f"{acc:.4f}", 
                     ha='center', va='bottom')
    
    plt.tight_layout()
    plt.suptitle(title, fontsize=16, y=1.02)
    plt.savefig("classifier_performance.png")
    plt.show()

def estimate_transition_matrix_anchor_points(X, S, num_classes=4, anchor_threshold=0.9):
    """
    Estimate transition matrix using anchor points method.
    
    Args:
        X: Features
        S: Noisy labels
        num_classes: Number of classes
        anchor_threshold: Confidence threshold for anchor points
        
    Returns:
        Estimated transition matrix
    """
    from sklearn.model_selection import train_test_split
    
    # Split data for initial model training
    X_train, X_val, S_train, S_val = train_test_split(X, S, test_size=0.2)
    
    # Create and train a base model
    with tf.device('/device:GPU:0'):
        base_model = models.Sequential([
            layers.Input(shape=X.shape[1:]),
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(128, activation='relu'),
            layers.Dense(num_classes, activation='softmax')
        ])
        
        base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
        base_model.fit(X_train, S_train, validation_data=(X_val, S_val), 
                       epochs=10, verbose=1)
        
        # Get predicted probabilities
        pred_probs = base_model.predict(X)
    
    # Initialize transition matrix
    T = np.zeros((num_classes, num_classes))
    
    # For each true class, find anchor points and estimate transition probabilities
    for i in range(num_classes):
        # Find anchor points: points where the model is very confident about class i
        anchor_idx = np.where(pred_probs[:, i] >= anchor_threshold)[0]
        
        if len(anchor_idx) > 0:
            # Count occurrences of each noisy label among anchor points
            for j in range(num_classes):
                class_j_count = np.sum(S[anchor_idx] == j)
                T[i, j] = class_j_count / len(anchor_idx)
        else:
            # If no anchor points found, use a fallback (identity or prior knowledge)
            T[i, i] = 0.7  # Assuming diagonal dominance as a prior
            T[i, (i+1) % num_classes] = 0.3  # Some off-diagonal probability
    
    # Ensure each row sums to 1
    row_sums = T.sum(axis=1, keepdims=True)
    T = T / row_sums
    
    return T

def generate_results_table(results):
    """
    Generate a formatted results table for the report.
    
    Args:
        results: Dictionary of results from run_evaluations
        
    Returns:
        Formatted string with results table
    """
    # Create header
    table = "| Dataset | Classifier | Accuracy (%) | Std. Dev. |\n"
    table += "|---------|------------|-------------|----------|\n"
    
    # Add rows
    for dataset_name, dataset_results in results.items():
        for i, (classifier_name, (mean_acc, std_acc)) in enumerate(dataset_results.items()):
            # First row of a dataset gets the dataset name
            if i == 0:
                table += f"| {dataset_name} | {classifier_name} | {mean_acc*100:.2f} | {std_acc*100:.2f} |\n"
            else:
                table += f"| | {classifier_name} | {mean_acc*100:.2f} | {std_acc*100:.2f} |\n"
    
    return table

def main():
    """
    Main execution function demonstrating the evaluation framework on GPU.
    """
    print("Starting evaluation framework on CUDA:5")
    print("======================================")
    
    # Load datasets
    print("Loading datasets...")
    
    # FashionMNIST 0.3 noise
    dataset1 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/FashionMNIST0.3.npz')
    Xtr1, Str1 = dataset1['X_tr'], dataset1['S_tr']
    Xts1, Yts1 = dataset1['X_ts'], dataset1['Y_ts']
    
    # Preprocess data (normalize pixel values)
    Xtr1 = Xtr1.astype('float32') / 255.0
    Xts1 = Xts1.astype('float32') / 255.0
    
    # FashionMNIST 0.6 noise
    dataset2 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/FashionMNIST0.6.npz')
    Xtr2, Str2 = dataset2['X_tr'], dataset2['S_tr']
    Xts2, Yts2 = dataset2['X_ts'], dataset2['Y_ts']
    
    # Preprocess data
    Xtr2 = Xtr2.astype('float32') / 255.0
    Xts2 = Xts2.astype('float32') / 255.0
    
    # CIFAR - unknown transition matrix
    dataset3 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/CIFAR10.npz')
    Xtr3, Str3 = dataset3['X_tr'], dataset3['S_tr']
    Xts3, Yts3 = dataset3['X_ts'], dataset3['Y_ts']
    
    # Preprocess data
    Xtr3 = Xtr3.astype('float32') / 255.0
    Xts3 = Xts3.astype('float32') / 255.0
    
    # Define transition matrices
    T_fashion_03 = np.array([
        [0.7, 0.3, 0, 0],
        [0, 0.7, 0.3, 0],
        [0, 0, 0.7, 0.3],
        [0.3, 0, 0, 0.7]
    ])
    
    T_fashion_06 = np.array([
        [0.4, 0.2, 0.2, 0.2],
        [0.2, 0.4, 0.2, 0.2],
        [0.2, 0.2, 0.4, 0.2],
        [0.2, 0.2, 0.2, 0.4]
    ])
    
    # Create transition matrix estimator for CIFAR
    print("Estimating transition matrix for CIFAR dataset...")
    T_cifar_estimated = estimate_transition_matrix_anchor_points(Xtr3, Str3)
    print("Estimated transition matrix for CIFAR:")
    print(T_cifar_estimated)
    
    # Organize datasets for evaluation
    # To save time during testing, you might want to only use a small subset of data
    # Comment out the datasets you don't want to evaluate
    datasets = {
        'FashionMNIST_0.3': (Xtr1, Str1, Xts1, Yts1),
        'FashionMNIST_0.6': (Xtr2, Str2, Xts2, Yts2),
        'CIFAR': (Xtr3, Str3, Xts3, Yts3)
    }
    
    # Define model builders
    def build_fashion_mnist_cnn():
        model = models.Sequential([
            layers.Reshape((28, 28, 1), input_shape=(28, 28)),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(4)  # 4 classes
        ])
        return model
    
    def build_cifar_cnn():
        model = models.Sequential([
            layers.Input(shape=(32, 32, 3)),
            layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.2),
            layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.3),
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(4)  # 4 classes
        ])
        return model
    
    # Create classifiers
    classifiers = [
        # Standard CNN without correction
        NoiseRobustClassifier(
            name="Standard CNN",
            model_builder=build_fashion_mnist_cnn,
            use_correction=False
        ),
        
        # Forward Correction with known transition matrix (for FashionMNIST_0.3)
        NoiseRobustClassifier(
            name="Forward Correction (T_0.3)",
            model_builder=build_fashion_mnist_cnn,
            transition_matrix=T_fashion_03,
            use_correction=True
        ),
        
        # Forward Correction with known transition matrix (for FashionMNIST_0.6)
        NoiseRobustClassifier(
            name="Forward Correction (T_0.6)",
            model_builder=build_fashion_mnist_cnn,
            transition_matrix=T_fashion_06,
            use_correction=True
        ),
        
        # Forward Correction with estimated transition matrix (for CIFAR)
        NoiseRobustClassifier(
            name="Forward Correction (T_estimated)",
            model_builder=build_cifar_cnn,
            transition_matrix=T_cifar_estimated,
            use_correction=True
        )
    ]
    
    # Run evaluations
    results = run_evaluations(datasets, classifiers, n_runs=3)
    
    # Visualize results
    visualize_results(results)
    
    # Generate results table for report
    table = generate_results_table(results)
    print("\nResults Table for Report:")
    print(table)
    
    # Save results to a file
    with open("results_table.md", "w") as f:
        f.write(table)
    
    print("\nEvaluation completed successfully!")

if __name__ == "__main__":
    main()

2025-03-14 23:27:10.696332: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 23:27:10.716372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-14 23:27:10.733565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-14 23:27:10.738847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 23:27:10.751676: I tensorflow/core/platform/cpu_feature_guar

No GPU found, using CPU instead
TensorFlow is using devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
TensorFlow CUDA available: True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
TensorFlow GPU available: False
Starting evaluation framework on CUDA:5
Loading datasets...


2025-03-14 23:27:26.134723: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-03-14 23:27:26.145112: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Estimating transition matrix for CIFAR dataset...
Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 1.3816 - val_loss: 1.2084
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 1.2410 - val_loss: 1.1491
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.1978 - val_loss: 1.1679
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 1.1900 - val_loss: 1.1504
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.1612 - val_loss: 1.1626
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.1722 - val_loss: 1.1485
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.1527 - val_loss: 1.1345
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.1431 - val_loss: 1.1375
Epoch 9/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.1373 - val_loss: 1.0967
Epoch 10/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.1245 - val_loss: 1.1224
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Estimated transition matrix for CIFAR:
[[0.7    

/home/aidar.alimbayev/anaconda3/envs/ml817/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.2760 - loss: 1.8664 - val_accuracy: 0.2433 - val_loss: 4.9364
Epoch 2/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.2868 - loss: 1.3984 - val_accuracy: 0.3512 - val_loss: 2.1334
Epoch 3/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.2921 - loss: 1.3914 - val_accuracy: 0.2406 - val_loss: 1.3948
Epoch 4/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.2650 - loss: 1.4020 - val_accuracy: 0.3392 - val_loss: 1.3863
Epoch 5/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.2977 - loss: 1.3909 - val_accuracy: 0.3388 - val_loss: 1.3863
Epoch 6/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.3000 - loss: 1.3895 - val_accuracy: 0.3385 - val_loss: 1.3863
Epoch 7/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.3058 - loss: 1.3863 - val_accuracy: 0.3385 - val_loss: 1.3863
Epoch 8/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.2987 - loss: 1.3867 - val_accuracy: 0.338

TypeError: len is not well defined for a symbolic Tensor (compile_loss/forward_correction_loss/Shape:0). Please call `x.shape` rather than `len(x)` for shape information.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os

# Set CUDA device
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class NoiseRobustClassifier:
    def __init__(self, name, model_builder, transition_matrix=None, use_correction=True):
        """
        A wrapper for noise-robust classifiers.
        
        Args:
            name: Name of the classifier for reporting
            model_builder: Function that returns a PyTorch model
            transition_matrix: The noise transition matrix if known
            use_correction: Whether to use transition matrix correction
        """
        self.name = name
        self.model_builder = model_builder
        self.transition_matrix = torch.tensor(transition_matrix).float().to(device) if transition_matrix is not None else None
        self.use_correction = use_correction
        self.model = None
        self.history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    def forward_correction_loss(self, y_pred, y_true):
        """Forward correction loss using transition matrix"""
        # Get predicted probabilities
        pred_probs = torch.softmax(y_pred, dim=1)
        
        # Apply transition matrix correction
        corrected_probs = torch.matmul(pred_probs, self.transition_matrix)
        
        # Calculate cross-entropy loss with corrected probabilities
        loss = nn.CrossEntropyLoss()(torch.log(corrected_probs + 1e-8), y_true)
        
        return loss
    
    def fit(self, X_train, S_train, X_val, S_val, epochs=30, batch_size=128):
        """Train the classifier on noisy data and validate."""
        # Convert data to PyTorch tensors and move to GPU
        X_train = torch.tensor(X_train).float().to(device)
        S_train = torch.tensor(S_train).long().to(device)
        X_val = torch.tensor(X_val).float().to(device)
        S_val = torch.tensor(S_val).long().to(device)
        
        # Build model and move to GPU
        self.model = self.model_builder().to(device)
        
        # Define optimizer
        optimizer = optim.Adam(self.model.parameters())
        
        # Define loss function
        criterion = self.forward_correction_loss if (self.transition_matrix is not None and self.use_correction) else nn.CrossEntropyLoss()
        
        best_val_loss = float('inf')
        patience_counter = 0
        best_state = None
        
        for epoch in range(epochs):
            # Training
            self.model.train()
            train_loss = 0
            correct = 0
            total = 0
            
            for i in range(0, len(X_train), batch_size):
                batch_X = X_train[i:i+batch_size]
                batch_S = S_train[i:i+batch_size]
                
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_S)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                _, predicted = outputs.max(1)
                total += batch_S.size(0)
                correct += predicted.eq(batch_S).sum().item()
            
            train_acc = correct / total
            train_loss = train_loss * batch_size / len(X_train)
            
            # Validation
            self.model.eval()
            val_loss = 0
            correct = 0
            total = 0
            
            with torch.no_grad():
                for i in range(0, len(X_val), batch_size):
                    batch_X = X_val[i:i+batch_size]
                    batch_S = S_val[i:i+batch_size]
                    
                    outputs = self.model(batch_X)
                    loss = criterion(outputs, batch_S)
                    
                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    total += batch_S.size(0)
                    correct += predicted.eq(batch_S).sum().item()
            
            val_acc = correct / total
            val_loss = val_loss * batch_size / len(X_val)
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_state = self.model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= 5:
                    self.model.load_state_dict(best_state)
                    break
            
            # Save history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_acc'].append(val_acc)
            
            print(f'Epoch {epoch+1}/{epochs}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')
        
        return self
    
    def evaluate(self, X_test, Y_test):
        """Evaluate the classifier on clean test data."""
        self.model.eval()
        
        # Convert data to PyTorch tensors and move to GPU
        X_test = torch.tensor(X_test).float().to(device)
        Y_test = torch.tensor(Y_test).long().to(device)
        
        with torch.no_grad():
            outputs = self.model(X_test)
            _, predicted = outputs.max(1)
            accuracy = predicted.eq(Y_test).float().mean().item()
        
        return accuracy

def evaluate_classifier(classifier, Xtr, Str, Xts, Yts, n_runs=3):
    """Evaluate a classifier multiple times with different random splits."""
    accuracies = []
    
    for i in range(n_runs):
        seed = 42 + i
        torch.manual_seed(seed)
        np.random.seed(seed)
        
        X_train, X_val, S_train, S_val = train_test_split(
            Xtr, Str, test_size=0.2, random_state=seed
        )
        
        print(f"Run {i+1}/{n_runs} - Training {classifier.name}...")
        
        classifier.fit(X_train, S_train, X_val, S_val)
        accuracy = classifier.evaluate(Xts, Yts)
        accuracies.append(accuracy)
        print(f"Run {i+1}/{n_runs} - Test accuracy: {accuracy:.4f}")
    
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    
    return mean_acc, std_acc

class FashionMNISTCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

class CIFARCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 3, 32, 32)
        x = self.dropout1(self.pool(self.relu(self.bn1(self.conv1(x)))))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(self.pool(self.relu(self.bn3(self.conv3(x)))))
        x = self.flatten(x)
        x = self.dropout3(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

def build_fashion_mnist_cnn():
    return FashionMNISTCNN()

def build_cifar_cnn():
    return CIFARCNN()

def run_evaluations(datasets, classifiers, n_runs=3):
    """
    Run evaluations for multiple classifiers on multiple datasets.
    
    Args:
        datasets: Dict of {dataset_name: (Xtr, Str, Xts, Yts)}
        classifiers: List of NoiseRobustClassifier instances
        n_runs: Number of runs with different random splits
        
    Returns:
        Dictionary of results {dataset_name: {classifier_name: (mean_acc, std_acc)}}
    """
    results = {}
    
    for dataset_name, (Xtr, Str, Xts, Yts) in datasets.items():
        print(f"\n=== Evaluating on dataset: {dataset_name} ===")
        dataset_results = {}
        
        for classifier in classifiers:
            print(f"\n--- Evaluating classifier: {classifier.name} ---")
            mean_acc, std_acc = evaluate_classifier(classifier, Xtr, Str, Xts, Yts, n_runs)
            dataset_results[classifier.name] = (mean_acc, std_acc)
            print(f"Final results - Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
        
        results[dataset_name] = dataset_results
    
    return results

def visualize_results(results, title="Classifier Performance Comparison"):
    """
    Create bar plots to visualize classifier performance across datasets.
    
    Args:
        results: Dictionary of results from run_evaluations
        title: Plot title
    """
    # Determine dimensions
    n_datasets = len(results)
    n_classifiers = len(next(iter(results.values())))
    
    # Set up the figure
    plt.figure(figsize=(12, 5 * n_datasets))
    
    # For each dataset
    for i, (dataset_name, dataset_results) in enumerate(results.items()):
        # Create subplot
        plt.subplot(n_datasets, 1, i+1)
        
        # Extract data for plotting
        classifiers = list(dataset_results.keys())
        accuracies = [dataset_results[clf][0] for clf in classifiers]  # mean
        errors = [dataset_results[clf][1] for clf in classifiers]      # std
        
        # Create bar plot
        bars = plt.bar(classifiers, accuracies, yerr=errors, capsize=10)
        
        # Add labels and grid
        plt.title(f"{dataset_name} - Test Accuracy")
        plt.ylabel("Accuracy")
        plt.ylim(0, 1.0)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45, ha='right')
        
        # Add accuracy values on top of bars
        for bar, acc in zip(bars, accuracies):
            plt.text(bar.get_x() + bar.get_width()/2, 
                     bar.get_height() + 0.01, 
                     f"{acc:.4f}", 
                     ha='center', va='bottom')
    
    plt.tight_layout()
    plt.suptitle(title, fontsize=16, y=1.02)
    plt.savefig("classifier_performance.png", bbox_inches='tight')
    plt.close()

def estimate_transition_matrix_anchor_points(X, S, num_classes=4, anchor_threshold=0.9):
    """
    Estimate transition matrix using anchor points method.
    
    Args:
        X: Features
        S: Noisy labels
        num_classes: Number of classes
        anchor_threshold: Confidence threshold for anchor points
        
    Returns:
        Estimated transition matrix
    """
    from sklearn.model_selection import train_test_split
    
    # Split data for initial model training
    X_train, X_val, S_train, S_val = train_test_split(X, S, test_size=0.2)
    
    # Create and train a base model
    base_model = CIFARCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(base_model.parameters())
    
    # Convert data to tensors
    X_train = torch.tensor(X_train).float().to(device)
    S_train = torch.tensor(S_train).long().to(device)
    X_val = torch.tensor(X_val).float().to(device)
    S_val = torch.tensor(S_val).long().to(device)
    X = torch.tensor(X).float().to(device)
    
    # Train base model
    for epoch in range(10):
        base_model.train()
        for i in range(0, len(X_train), 128):
            batch_X = X_train[i:i+128]
            batch_S = S_train[i:i+128]
            
            optimizer.zero_grad()
            outputs = base_model(batch_X)
            loss = criterion(outputs, batch_S)
            loss.backward()
            optimizer.step()
    
    # Get predicted probabilities
    base_model.eval()
    with torch.no_grad():
        pred_probs = []
        for i in range(0, len(X), 128):
            batch_X = X[i:i+128]
            outputs = base_model(batch_X)
            probs = torch.softmax(outputs, dim=1)
            pred_probs.append(probs)
        pred_probs = torch.cat(pred_probs, dim=0).cpu().numpy()
    
    # Initialize transition matrix
    T = np.zeros((num_classes, num_classes))
    
    # For each true class, find anchor points and estimate transition probabilities
    for i in range(num_classes):
        # Find anchor points: points where the model is very confident about class i
        anchor_idx = np.where(pred_probs[:, i] >= anchor_threshold)[0]
        
        if len(anchor_idx) > 0:
            # Count occurrences of each noisy label among anchor points
            for j in range(num_classes):
                class_j_count = np.sum(S[anchor_idx] == j)
                T[i, j] = class_j_count / len(anchor_idx)
        else:
            # If no anchor points found, use a fallback (identity or prior knowledge)
            T[i, i] = 0.7  # Assuming diagonal dominance as a prior
            T[i, (i+1) % num_classes] = 0.3  # Some off-diagonal probability
    
    # Ensure each row sums to 1
    row_sums = T.sum(axis=1, keepdims=True)
    T = T / row_sums
    
    return T

def generate_results_table(results):
    """
    Generate a formatted results table for the report.
    
    Args:
        results: Dictionary of results from run_evaluations
        
    Returns:
        Formatted string with results table
    """
    # Create header
    table = "| Dataset | Classifier | Accuracy (%) | Std. Dev. |\n"
    table += "|---------|------------|-------------|----------|\n"
    
    # Add rows
    for dataset_name, dataset_results in results.items():
        for i, (classifier_name, (mean_acc, std_acc)) in enumerate(dataset_results.items()):
            # First row of a dataset gets the dataset name
            if i == 0:
                table += f"| {dataset_name} | {classifier_name} | {mean_acc*100:.2f} | {std_acc*100:.2f} |\n"
            else:
                table += f"| | {classifier_name} | {mean_acc*100:.2f} | {std_acc*100:.2f} |\n"
    
    return table

def main():
    """
    Main execution function demonstrating the evaluation framework on GPU.
    """
    print("Starting evaluation framework on CUDA:5")
    print("======================================")
    
    # Load datasets
    print("Loading datasets...")
    
    # FashionMNIST 0.3 noise
    dataset1 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/FashionMNIST0.3.npz')
    Xtr1, Str1 = dataset1['X_tr'], dataset1['S_tr']
    Xts1, Yts1 = dataset1['X_ts'], dataset1['Y_ts']
    
    # Preprocess data (normalize pixel values)
    Xtr1 = Xtr1.astype('float32') / 255.0
    Xts1 = Xts1.astype('float32') / 255.0
    
    # FashionMNIST 0.6 noise
    dataset2 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/FashionMNIST0.6.npz')
    Xtr2, Str2 = dataset2['X_tr'], dataset2['S_tr']
    Xts2, Yts2 = dataset2['X_ts'], dataset2['Y_ts']
    
    # Preprocess data
    Xtr2 = Xtr2.astype('float32') / 255.0
    Xts2 = Xts2.astype('float32') / 255.0
    
    # CIFAR - unknown transition matrix
    dataset3 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/CIFAR10.npz')
    Xtr3, Str3 = dataset3['X_tr'], dataset3['S_tr']
    Xts3, Yts3 = dataset3['X_ts'], dataset3['Y_ts']
    
    # Preprocess data
    Xtr3 = Xtr3.astype('float32') / 255.0
    Xts3 = Xts3.astype('float32') / 255.0
    
    # Define transition matrices
    T_fashion_03 = np.array([
        [0.7, 0.3, 0, 0],
        [0, 0.7, 0.3, 0],
        [0, 0, 0.7, 0.3],
        [0.3, 0, 0, 0.7]
    ])
    
    T_fashion_06 = np.array([
        [0.4, 0.2, 0.2, 0.2],
        [0.2, 0.4, 0.2, 0.2],
        [0.2, 0.2, 0.4, 0.2],
        [0.2, 0.2, 0.2, 0.4]
    ])
    
    # Create transition matrix estimator for CIFAR
    print("Estimating transition matrix for CIFAR dataset...")
    T_cifar_estimated = estimate_transition_matrix_anchor_points(Xtr3, Str3)
    print("Estimated transition matrix for CIFAR:")
    print(T_cifar_estimated)
    
    # Organize datasets for evaluation
    # To save time during testing, you might want to only use a small subset of data
    # Comment out the datasets you don't want to evaluate
    datasets = {
        'FashionMNIST_0.3': (Xtr1, Str1, Xts1, Yts1),
        'FashionMNIST_0.6': (Xtr2, Str2, Xts2, Yts2),
        'CIFAR': (Xtr3, Str3, Xts3, Yts3)
    }
    

    
    # Create classifiers
    classifiers = [
        # Standard CNN without correction
        NoiseRobustClassifier(
            name="Standard CNN",
            model_builder=build_fashion_mnist_cnn,
            use_correction=False
        ),
        
        # Forward Correction with known transition matrix (for FashionMNIST_0.3)
        NoiseRobustClassifier(
            name="Forward Correction (T_0.3)",
            model_builder=build_fashion_mnist_cnn,
            transition_matrix=T_fashion_03,
            use_correction=True
        ),
        
        # Forward Correction with known transition matrix (for FashionMNIST_0.6)
        NoiseRobustClassifier(
            name="Forward Correction (T_0.6)",
            model_builder=build_fashion_mnist_cnn,
            transition_matrix=T_fashion_06,
            use_correction=True
        ),
        
        # Forward Correction with estimated transition matrix (for CIFAR)
        NoiseRobustClassifier(
            name="Forward Correction (T_estimated)",
            model_builder=build_cifar_cnn,
            transition_matrix=T_cifar_estimated,
            use_correction=True
        )
    ]
    
    # Run evaluations
    results = run_evaluations(datasets, classifiers, n_runs=3)
    
    # Visualize results
    visualize_results(results)
    
    # Generate results table for report
    table = generate_results_table(results)
    print("\nResults Table for Report:")
    print(table)
    
    # Save results to a file
    with open("results_table.md", "w") as f:
        f.write(table)
    
    print("\nEvaluation completed successfully!")

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os

# Set CUDA device
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class NoiseRobustClassifier:
    def __init__(self, name, model_builder, transition_matrix=None, use_correction=True):
        """
        A wrapper for noise-robust classifiers.
        
        Args:
            name: Name of the classifier for reporting
            model_builder: Function that returns a PyTorch model
            transition_matrix: The noise transition matrix if known
            use_correction: Whether to use transition matrix correction
        """
        self.name = name
        self.model_builder = model_builder
        self.transition_matrix = torch.tensor(transition_matrix).float().to(device) if transition_matrix is not None else None
        self.use_correction = use_correction
        self.model = None
        self.history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    def forward_correction_loss(self, y_pred, y_true):
        """Forward correction loss using transition matrix"""
        # Get predicted probabilities
        pred_probs = torch.softmax(y_pred, dim=1)
        
        # Apply transition matrix correction
        corrected_probs = torch.matmul(pred_probs, self.transition_matrix)
        
        # Calculate cross-entropy loss with corrected probabilities
        loss = nn.CrossEntropyLoss()(torch.log(corrected_probs + 1e-8), y_true)
        
        return loss
    
    def fit(self, X_train, S_train, X_val, S_val, epochs=30, batch_size=128):
        """Train the classifier on noisy data and validate."""
        # Convert data to PyTorch tensors and move to GPU
        X_train = torch.tensor(X_train).float().to(device)
        S_train = torch.tensor(S_train).long().to(device)
        X_val = torch.tensor(X_val).float().to(device)
        S_val = torch.tensor(S_val).long().to(device)
        
        # Build model and move to GPU
        self.model = self.model_builder().to(device)
        
        # Define optimizer
        optimizer = optim.Adam(self.model.parameters())
        
        # Define loss function
        criterion = self.forward_correction_loss if (self.transition_matrix is not None and self.use_correction) else nn.CrossEntropyLoss()
        
        best_val_loss = float('inf')
        patience_counter = 0
        best_state = None
        
        for epoch in range(epochs):
            # Training
            self.model.train()
            train_loss = 0
            correct = 0
            total = 0
            
            for i in range(0, len(X_train), batch_size):
                batch_X = X_train[i:i+batch_size]
                batch_S = S_train[i:i+batch_size]
                
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_S)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                _, predicted = outputs.max(1)
                total += batch_S.size(0)
                correct += predicted.eq(batch_S).sum().item()
            
            train_acc = correct / total
            train_loss = train_loss * batch_size / len(X_train)
            
            # Validation
            self.model.eval()
            val_loss = 0
            correct = 0
            total = 0
            
            with torch.no_grad():
                for i in range(0, len(X_val), batch_size):
                    batch_X = X_val[i:i+batch_size]
                    batch_S = S_val[i:i+batch_size]
                    
                    outputs = self.model(batch_X)
                    loss = criterion(outputs, batch_S)
                    
                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    total += batch_S.size(0)
                    correct += predicted.eq(batch_S).sum().item()
            
            val_acc = correct / total
            val_loss = val_loss * batch_size / len(X_val)
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_state = self.model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= 5:
                    self.model.load_state_dict(best_state)
                    break
            
            # Save history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_acc'].append(val_acc)
            
            print(f'Epoch {epoch+1}/{epochs}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')
        
        return self
    
    def evaluate(self, X_test, Y_test):
        """Evaluate the classifier on clean test data."""
        self.model.eval()
        
        # Convert data to PyTorch tensors and move to GPU
        X_test = torch.tensor(X_test).float().to(device)
        Y_test = torch.tensor(Y_test).long().to(device)
        
        with torch.no_grad():
            outputs = self.model(X_test)
            _, predicted = outputs.max(1)
            accuracy = predicted.eq(Y_test).float().mean().item()
        
        return accuracy

def evaluate_classifier(classifier, Xtr, Str, Xts, Yts, n_runs=3):
    """Evaluate a classifier multiple times with different random splits."""
    accuracies = []
    
    for i in range(n_runs):
        seed = 42 + i
        torch.manual_seed(seed)
        np.random.seed(seed)
        
        X_train, X_val, S_train, S_val = train_test_split(
            Xtr, Str, test_size=0.2, random_state=seed
        )
        
        print(f"Run {i+1}/{n_runs} - Training {classifier.name}...")
        
        classifier.fit(X_train, S_train, X_val, S_val)
        accuracy = classifier.evaluate(Xts, Yts)
        accuracies.append(accuracy)
        print(f"Run {i+1}/{n_runs} - Test accuracy: {accuracy:.4f}")
    
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    
    return mean_acc, std_acc

class FashionMNISTCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

class CIFARCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(2)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 3, 32, 32)
        x = self.dropout1(self.pool(self.relu(self.bn1(self.conv1(x)))))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(self.pool(self.relu(self.bn3(self.conv3(x)))))
        x = self.flatten(x)
        x = self.dropout3(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

def build_fashion_mnist_cnn():
    return FashionMNISTCNN()

def build_cifar_cnn():
    return CIFARCNN()

def run_evaluations(datasets, classifiers, n_runs=3):
    """
    Run evaluations for multiple classifiers on multiple datasets.
    
    Args:
        datasets: Dict of {dataset_name: (Xtr, Str, Xts, Yts)}
        classifiers: List of NoiseRobustClassifier instances
        n_runs: Number of runs with different random splits
        
    Returns:
        Dictionary of results {dataset_name: {classifier_name: (mean_acc, std_acc)}}
    """
    results = {}
    
    for dataset_name, (Xtr, Str, Xts, Yts) in datasets.items():
        print(f"\n=== Evaluating on dataset: {dataset_name} ===")
        dataset_results = {}
        
        for classifier in classifiers:
            print(f"\n--- Evaluating classifier: {classifier.name} ---")
            mean_acc, std_acc = evaluate_classifier(classifier, Xtr, Str, Xts, Yts, n_runs)
            dataset_results[classifier.name] = (mean_acc, std_acc)
            print(f"Final results - Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
        
        results[dataset_name] = dataset_results
    
    return results

def visualize_results(results, title="Classifier Performance Comparison"):
    """
    Create bar plots to visualize classifier performance across datasets.
    
    Args:
        results: Dictionary of results from run_evaluations
        title: Plot title
    """
    # Determine dimensions
    n_datasets = len(results)
    n_classifiers = len(next(iter(results.values())))
    
    # Set up the figure
    plt.figure(figsize=(12, 5 * n_datasets))
    
    # For each dataset
    for i, (dataset_name, dataset_results) in enumerate(results.items()):
        # Create subplot
        plt.subplot(n_datasets, 1, i+1)
        
        # Extract data for plotting
        classifiers = list(dataset_results.keys())
        accuracies = [dataset_results[clf][0] for clf in classifiers]  # mean
        errors = [dataset_results[clf][1] for clf in classifiers]      # std
        
        # Create bar plot
        bars = plt.bar(classifiers, accuracies, yerr=errors, capsize=10)
        
        # Add labels and grid
        plt.title(f"{dataset_name} - Test Accuracy")
        plt.ylabel("Accuracy")
        plt.ylim(0, 1.0)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45, ha='right')
        
        # Add accuracy values on top of bars
        for bar, acc in zip(bars, accuracies):
            plt.text(bar.get_x() + bar.get_width()/2, 
                     bar.get_height() + 0.01, 
                     f"{acc:.4f}", 
                     ha='center', va='bottom')
    
    plt.tight_layout()
    plt.suptitle(title, fontsize=16, y=1.02)
    plt.savefig("classifier_performance.png", bbox_inches='tight')
    plt.close()

def estimate_transition_matrix_anchor_points(X, S, num_classes=4, anchor_threshold=0.9):
    """
    Estimate transition matrix using anchor points method.
    
    Args:
        X: Features
        S: Noisy labels
        num_classes: Number of classes
        anchor_threshold: Confidence threshold for anchor points
        
    Returns:
        Estimated transition matrix
    """
    from sklearn.model_selection import train_test_split
    
    # Split data for initial model training
    X_train, X_val, S_train, S_val = train_test_split(X, S, test_size=0.2)
    
    # Create and train a base model
    base_model = CIFARCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(base_model.parameters())
    
    # Convert data to tensors
    X_train = torch.tensor(X_train).float().to(device)
    S_train = torch.tensor(S_train).long().to(device)
    X_val = torch.tensor(X_val).float().to(device)
    S_val = torch.tensor(S_val).long().to(device)
    X = torch.tensor(X).float().to(device)
    
    # Train base model
    for epoch in range(10):
        base_model.train()
        for i in range(0, len(X_train), 128):
            batch_X = X_train[i:i+128]
            batch_S = S_train[i:i+128]
            
            optimizer.zero_grad()
            outputs = base_model(batch_X)
            loss = criterion(outputs, batch_S)
            loss.backward()
            optimizer.step()
    
    # Get predicted probabilities
    base_model.eval()
    with torch.no_grad():
        pred_probs = []
        for i in range(0, len(X), 128):
            batch_X = X[i:i+128]
            outputs = base_model(batch_X)
            probs = torch.softmax(outputs, dim=1)
            pred_probs.append(probs)
        pred_probs = torch.cat(pred_probs, dim=0).cpu().numpy()
    
    # Initialize transition matrix
    T = np.zeros((num_classes, num_classes))
    
    # For each true class, find anchor points and estimate transition probabilities
    for i in range(num_classes):
        # Find anchor points: points where the model is very confident about class i
        anchor_idx = np.where(pred_probs[:, i] >= anchor_threshold)[0]
        
        if len(anchor_idx) > 0:
            # Count occurrences of each noisy label among anchor points
            for j in range(num_classes):
                class_j_count = np.sum(S[anchor_idx] == j)
                T[i, j] = class_j_count / len(anchor_idx)
        else:
            # If no anchor points found, use a fallback (identity or prior knowledge)
            T[i, i] = 0.7  # Assuming diagonal dominance as a prior
            T[i, (i+1) % num_classes] = 0.3  # Some off-diagonal probability
    
    # Ensure each row sums to 1
    row_sums = T.sum(axis=1, keepdims=True)
    T = T / row_sums
    
    return T

def generate_results_table(results):
    """
    Generate a formatted results table for the report.
    
    Args:
        results: Dictionary of results from run_evaluations
        
    Returns:
        Formatted string with results table
    """
    # Create header
    table = "| Dataset | Classifier | Accuracy (%) | Std. Dev. |\n"
    table += "|---------|------------|-------------|----------|\n"
    
    # Add rows
    for dataset_name, dataset_results in results.items():
        for i, (classifier_name, (mean_acc, std_acc)) in enumerate(dataset_results.items()):
            # First row of a dataset gets the dataset name
            if i == 0:
                table += f"| {dataset_name} | {classifier_name} | {mean_acc*100:.2f} | {std_acc*100:.2f} |\n"
            else:
                table += f"| | {classifier_name} | {mean_acc*100:.2f} | {std_acc*100:.2f} |\n"
    
    return table

def main():
    """
    Main execution function demonstrating the evaluation framework on GPU.
    """
    print("Starting evaluation framework on CUDA:5")
    print("======================================")
    
    # Load datasets
    print("Loading datasets...")
    
    # FashionMNIST 0.3 noise
    dataset1 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/FashionMNIST0.3.npz')
    Xtr1, Str1 = dataset1['X_tr'], dataset1['S_tr']
    Xts1, Yts1 = dataset1['X_ts'], dataset1['Y_ts']
    
    # Preprocess data (normalize pixel values)
    Xtr1 = Xtr1.astype('float32') / 255.0
    Xts1 = Xts1.astype('float32') / 255.0
    
    # FashionMNIST 0.6 noise
    dataset2 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/FashionMNIST0.6.npz')
    Xtr2, Str2 = dataset2['X_tr'], dataset2['S_tr']
    Xts2, Yts2 = dataset2['X_ts'], dataset2['Y_ts']
    
    # Preprocess data
    Xtr2 = Xtr2.astype('float32') / 255.0
    Xts2 = Xts2.astype('float32') / 255.0
    
    # CIFAR - unknown transition matrix
    dataset3 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/CIFAR10.npz')
    Xtr3, Str3 = dataset3['X_tr'], dataset3['S_tr']
    Xts3, Yts3 = dataset3['X_ts'], dataset3['Y_ts']
    
    # Preprocess data
    Xtr3 = Xtr3.astype('float32') / 255.0
    Xts3 = Xts3.astype('float32') / 255.0
    
    # Define transition matrices
    T_fashion_03 = np.array([
        [0.7, 0.3, 0, 0],
        [0, 0.7, 0.3, 0],
        [0, 0, 0.7, 0.3],
        [0.3, 0, 0, 0.7]
    ])
    
    T_fashion_06 = np.array([
        [0.4, 0.2, 0.2, 0.2],
        [0.2, 0.4, 0.2, 0.2],
        [0.2, 0.2, 0.4, 0.2],
        [0.2, 0.2, 0.2, 0.4]
    ])
    
    # Create transition matrix estimator for CIFAR
    print("Estimating transition matrix for CIFAR dataset...")
    T_cifar_estimated = estimate_transition_matrix_anchor_points(Xtr3, Str3)
    print("Estimated transition matrix for CIFAR:")
    print(T_cifar_estimated)
    
    # Organize datasets for evaluation
    # To save time during testing, you might want to only use a small subset of data
    # Comment out the datasets you don't want to evaluate
    datasets = {
        'FashionMNIST_0.3': (Xtr1, Str1, Xts1, Yts1),
        'FashionMNIST_0.6': (Xtr2, Str2, Xts2, Yts2),
        'CIFAR': (Xtr3, Str3, Xts3, Yts3)
    }
    

    
    # Create classifiers
    classifiers = [
        # Standard CNN without correction
        NoiseRobustClassifier(
            name="Standard CNN",
            model_builder=build_fashion_mnist_cnn,
            use_correction=False
        ),
        
        # Forward Correction with known transition matrix (for FashionMNIST_0.3)
        NoiseRobustClassifier(
            name="Forward Correction (T_0.3)",
            model_builder=build_fashion_mnist_cnn,
            transition_matrix=T_fashion_03,
            use_correction=True
        ),
        
        # Forward Correction with known transition matrix (for FashionMNIST_0.6)
        NoiseRobustClassifier(
            name="Forward Correction (T_0.6)",
            model_builder=build_fashion_mnist_cnn,
            transition_matrix=T_fashion_06,
            use_correction=True
        ),
        
        # Forward Correction with estimated transition matrix (for CIFAR)
        NoiseRobustClassifier(
            name="Forward Correction (T_estimated)",
            model_builder=build_cifar_cnn,
            transition_matrix=T_cifar_estimated,
            use_correction=True
        )
    ]
    
    # Run evaluations
    results = run_evaluations(datasets, classifiers, n_runs=3)
    
    # Visualize results
    visualize_results(results)
    
    # Generate results table for report
    table = generate_results_table(results)
    print("\nResults Table for Report:")
    print(table)
    
    # Save results to a file
    with open("results_table.md", "w") as f:
        f.write(table)
    
    print("\nEvaluation completed successfully!")

if __name__ == "__main__":
    main()


Using device: cuda:6
Starting evaluation framework on CUDA:5
Loading datasets...
Estimating transition matrix for CIFAR dataset...
Estimated transition matrix for CIFAR:
[[0.7        0.3        0.         0.        ]
 [0.         0.89261745 0.10738255 0.        ]
 [0.         0.         0.7        0.3       ]
 [0.3        0.         0.         0.7       ]]

=== Evaluating on dataset: FashionMNIST_0.3 ===

--- Evaluating classifier: Standard CNN ---
Run 1/3 - Training Standard CNN...
Epoch 1/30: train_loss=0.9000, val_loss=0.7531, train_acc=0.6093, val_acc=0.6540
Epoch 2/30: train_loss=0.7672, val_loss=0.7338, train_acc=0.6535, val_acc=0.6594
Epoch 3/30: train_loss=0.7492, val_loss=0.7303, train_acc=0.6614, val_acc=0.6619
Epoch 4/30: train_loss=0.7406, val_loss=0.7357, train_acc=0.6632, val_acc=0.6619
Epoch 5/30: train_loss=0.7302, val_loss=0.7279, train_acc=0.6659, val_acc=0.6629
Epoch 6/30: train_loss=0.7250, val_loss=0.7161, train_acc=0.6643, val_acc=0.6652
Epoch 7/30: train_loss=0.7

RuntimeError: shape '[-1, 3, 32, 32]' is invalid for input of size 100352

In [35]:
# Find transition matrix for CIFAR10

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CIFAR
dataset3 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/CIFAR10.npz')
Xtr3, Str3 = dataset3['X_tr'], dataset3['S_tr']
Xts3, Yts3 = dataset3['X_ts'], dataset3['Y_ts']

# Normalize data
Xtr3 = Xtr3.astype('float32') / 255.0
Xts3 = Xts3.astype('float32') / 255.0

# Transpose data to match PyTorch's expected format (N, C, H, W)
Xtr3 = np.transpose(Xtr3, (0, 3, 1, 2))
Xts3 = np.transpose(Xts3, (0, 3, 1, 2))

def estimate_transition_matrix_anchor_points(Xtr, Str, Xts, Yts, num_classes=4, anchor_threshold=0.9):
    """Estimate transition matrix using anchor points method."""
    X_train, X_val, S_train, S_val = train_test_split(Xtr, Str, test_size=0.2)
    
    # Create base model (ResNet-18)
    class BasicBlock(nn.Module):
        expansion = 1
        def __init__(self, in_planes, planes, stride=1):
            super(BasicBlock, self).__init__()
            self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(planes)
            self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn2 = nn.BatchNorm2d(planes)

            self.shortcut = nn.Sequential()
            if stride != 1 or in_planes != self.expansion*planes:
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(self.expansion*planes)
                )

        def forward(self, x):
            out = torch.relu(self.bn1(self.conv1(x)))
            out = self.bn2(self.conv2(out))
            out += self.shortcut(x)
            out = torch.relu(out)
            return out

    class ResNet(nn.Module):
        def __init__(self, block, num_blocks, num_classes=4):
            super(ResNet, self).__init__()
            self.in_planes = 64

            self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(64)
            self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
            self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
            self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
            self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
            self.linear = nn.Linear(512*block.expansion, num_classes)

        def _make_layer(self, block, planes, num_blocks, stride):
            strides = [stride] + [1]*(num_blocks-1)
            layers = []
            for stride in strides:
                layers.append(block(self.in_planes, planes, stride))
                self.in_planes = planes * block.expansion
            return nn.Sequential(*layers)

        def forward(self, x):
            out = torch.relu(self.bn1(self.conv1(x)))
            out = self.layer1(out)
            out = self.layer2(out)
            out = self.layer3(out)
            out = self.layer4(out)
            out = torch.nn.functional.avg_pool2d(out, 4)
            out = out.view(out.size(0), -1)
            out = self.linear(out)
            return out

    base_model = ResNet(BasicBlock, [2,2,2,2], num_classes=num_classes).to(device)
    
    # Train base model
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32).to(device),
                                torch.tensor(S_train, dtype=torch.long).to(device))
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(base_model.parameters())
    
    for epoch in range(10):
        base_model.train()
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = base_model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
    
    # Get predicted probabilities
    base_model.eval()
    with torch.no_grad():
        # Process in batches to avoid memory issues
        batch_size = 128
        pred_probs = []
        for i in range(0, len(Xtr), batch_size):
            batch = torch.tensor(Xtr[i:i+batch_size], dtype=torch.float32).to(device)
            pred_probs.append(torch.softmax(base_model(batch), dim=1).cpu().numpy())
        pred_probs = np.concatenate(pred_probs, axis=0)
        
        # Evaluate accuracy on test set
        test_preds = []
        for i in range(0, len(Xts), batch_size):
            batch = torch.tensor(Xts[i:i+batch_size], dtype=torch.float32).to(device)
            test_preds.append(torch.argmax(base_model(batch), dim=1).cpu().numpy())
        test_preds = np.concatenate(test_preds)
        test_accuracy = np.mean(test_preds == Yts)
        print(f"Base model test accuracy: {test_accuracy:.4f}")
    
    # Initialize transition matrix
    T = np.zeros((num_classes, num_classes))
    
    # Estimate transition probabilities
    for i in range(num_classes):
        anchor_idx = np.where(pred_probs[:, i] >= anchor_threshold)[0]
        
        if len(anchor_idx) > 0:
            for j in range(num_classes):
                T[i, j] = np.mean(Str[anchor_idx] == j)
        else:
            T[i, i] = 0.7
            T[i, (i+1) % num_classes] = 0.3
    
    # Normalize rows
    T = T / T.sum(axis=1, keepdims=True)
    
    return T

# Estimate transition matrix for CIFAR
print("Estimating transition matrix for CIFAR dataset...")
T_cifar_estimated = estimate_transition_matrix_anchor_points(Xtr3, Str3, Xts3, Yts3, num_classes=4, anchor_threshold=0.9)
print("Estimated transition matrix for CIFAR:")
print(T_cifar_estimated)

Estimating transition matrix for CIFAR dataset...
Base model test accuracy: 0.8167
Estimated transition matrix for CIFAR:
[[0.94202899 0.04057971 0.00869565 0.00869565]
 [0.02216066 0.92520776 0.05124654 0.00138504]
 [0.00257732 0.         0.95360825 0.04381443]
 [0.0620438  0.         0.00547445 0.93248175]]


In [31]:
# Classification with anchor points
Xtr3 = Xtr3.astype('float32') / 255.0
Xts3 = Xts3.astype('float32') / 255.0
num_classes = 4

# Estimate transition matrix
T_cifar_estimated = estimate_transition_matrix_anchor_points(Xtr3, Str3, Xts3, Yts3, num_classes=4, anchor_threshold=0.9)

# Create base model (ResNet-18)
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=4):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.nn.functional.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

base_model = ResNet(BasicBlock, [2,2,2,2], num_classes=num_classes).to(device)
    
# Evaluate accuracy on test set
X_test_tensor = torch.tensor(Xts3, dtype=torch.float32).to(device)
test_outputs = base_model(X_test_tensor)
test_preds = torch.argmax(test_outputs, dim=1).cpu().numpy()
test_accuracy = np.mean(test_preds == Yts3)
print(f"Base model test accuracy: {test_accuracy:.4f}")
    

Base model test accuracy: 0.8300
Base model test accuracy: 0.2557


In [ ]:
# Find transition matrix for CIFAR10

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CIFAR
dataset3 = np.load('/home/aidar.alimbayev/Documents/projects_aidar/ml814_lab/labs/datasets/CIFAR10.npz')
Xtr3, Str3 = dataset3['X_tr'], dataset3['S_tr']
Xts3, Yts3 = dataset3['X_ts'], dataset3['Y_ts']

# Normalize data
Xtr3 = Xtr3.astype('float32') / 255.0
Xts3 = Xts3.astype('float32') / 255.0

# Transpose data to match PyTorch's expected format (N, C, H, W)
Xtr3 = np.transpose(Xtr3, (0, 3, 1, 2))
Xts3 = np.transpose(Xts3, (0, 3, 1, 2))

def estimate_transition_matrix_anchor_points(Xtr, Str, Xts, Yts, num_classes=4, anchor_threshold=0.9, num_runs=3):
    """Estimate transition matrix using anchor points method."""
    accuracies = []
    
    for run in range(num_runs):
        print(f"\nRun {run+1}/{num_runs}")
        X_train, X_val, S_train, S_val = train_test_split(Xtr, Str, test_size=0.2)
        
        # Create base model (ResNet-18)
        class BasicBlock(nn.Module):
            expansion = 1
            def __init__(self, in_planes, planes, stride=1):
                super(BasicBlock, self).__init__()
                self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
                self.bn1 = nn.BatchNorm2d(planes)
                self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
                self.bn2 = nn.BatchNorm2d(planes)

                self.shortcut = nn.Sequential()
                if stride != 1 or in_planes != self.expansion*planes:
                    self.shortcut = nn.Sequential(
                        nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                        nn.BatchNorm2d(self.expansion*planes)
                    )

            def forward(self, x):
                out = torch.relu(self.bn1(self.conv1(x)))
                out = self.bn2(self.conv2(out))
                out += self.shortcut(x)
                out = torch.relu(out)
                return out

        class ResNet(nn.Module):
            def __init__(self, block, num_blocks, num_classes=4):
                super(ResNet, self).__init__()
                self.in_planes = 64

                self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
                self.bn1 = nn.BatchNorm2d(64)
                self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
                self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
                self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
                self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
                self.linear = nn.Linear(512*block.expansion, num_classes)

            def _make_layer(self, block, planes, num_blocks, stride):
                strides = [stride] + [1]*(num_blocks-1)
                layers = []
                for stride in strides:
                    layers.append(block(self.in_planes, planes, stride))
                    self.in_planes = planes * block.expansion
                return nn.Sequential(*layers)

            def forward(self, x):
                out = torch.relu(self.bn1(self.conv1(x)))
                out = self.layer1(out)
                out = self.layer2(out)
                out = self.layer3(out)
                out = self.layer4(out)
                out = torch.nn.functional.avg_pool2d(out, 4)
                out = out.view(out.size(0), -1)
                out = self.linear(out)
                return out

        # ResNet-18
        base_model = ResNet(BasicBlock, [2,2,2,2], num_classes=num_classes).to(device)
        
        # Train base model
        train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32).to(device),
                                    torch.tensor(S_train, dtype=torch.long).to(device))
        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(base_model.parameters())
        
        for epoch in range(10):
            base_model.train()
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = base_model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
        
        # Get predicted probabilities
        base_model.eval()
        with torch.no_grad():
            # Process in batches to avoid memory issues
            batch_size = 128
            pred_probs = []
            for i in range(0, len(Xtr), batch_size):
                batch = torch.tensor(Xtr[i:i+batch_size], dtype=torch.float32).to(device)
                pred_probs.append(torch.softmax(base_model(batch), dim=1).cpu().numpy())
            pred_probs = np.concatenate(pred_probs, axis=0)
            
            # Evaluate accuracy on test set
            test_preds = []
            for i in range(0, len(Xts), batch_size):
                batch = torch.tensor(Xts[i:i+batch_size], dtype=torch.float32).to(device)
                test_preds.append(torch.argmax(base_model(batch), dim=1).cpu().numpy())
            test_preds = np.concatenate(test_preds)
            test_accuracy = np.mean(test_preds == Yts)
            print(f"Base model test accuracy: {test_accuracy:.4f}")
            accuracies.append(test_accuracy)
    
    # Calculate mean and std of accuracies
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    print(f"\nAccuracy across {num_runs} runs: {mean_acc:.4f} ± {std_acc:.4f}")
    
    # Initialize transition matrix
    T = np.zeros((num_classes, num_classes))
    
    # Estimate transition probabilities
    for i in range(num_classes):
        anchor_idx = np.where(pred_probs[:, i] >= anchor_threshold)[0]
        
        if len(anchor_idx) > 0:
            for j in range(num_classes):
                T[i, j] = np.mean(Str[anchor_idx] == j)
        else:
            T[i, i] = 0.7
            T[i, (i+1) % num_classes] = 0.3
    
    # Normalize rows
    T = T / T.sum(axis=1, keepdims=True)
    
    return T

# Estimate transition matrix for CIFAR
print("Estimating transition matrix for CIFAR dataset...")
T_cifar_estimated = estimate_transition_matrix_anchor_points(Xtr3, Str3, Xts3, Yts3, num_classes=4, anchor_threshold=0.9)
print("Estimated transition matrix for CIFAR:")
print(T_cifar_estimated)

Estimating transition matrix for CIFAR dataset...

Run 1/3
